In [44]:
import os
import pandas as pd
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import re
from datetime import date
import datetime
import time
import numpy as np
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
from time import sleep

# Folder path where the DataFrames are located
folder_path = '../data/raw_data/england/'

# Initialize a list to store the DataFrames
dfs = []

# Iterate through the files in the folder
for filename in os.listdir(folder_path):
    if filename.endswith('.csv'):  # Assuming the files are in CSV format
        file_path = os.path.join(folder_path, filename)
        df = pd.read_csv(file_path)
        if int(df.Date.iloc[1].split('/')[2]) < 2000:
            df['Season'] = int(df.Date.iloc[1].split('/')[2]) +2000
        else:
            df['Season'] = int(df.Date.iloc[1].split('/')[2])
        dfs.append(df)

# Concatenate the DataFrames
merged_df = pd.concat(dfs, ignore_index=True, sort=False).drop_duplicates()

# Rename variables to be consistent with Swedish Df
merged_df.rename(columns={'Div':'League', 'HomeTeam':'Home', 'AwayTeam':'Away',
                         'FTHG':'HG','FTAG':'AG', 'FTR':'Res',
                         'PSH':'PH', 'PSD':'PD', 'PSA': 'PA'}, inplace=True)

merged_df['Country'] = 'England'
# merged_df now contains the union of all DataFrames, handling additional or missing columns

#Keep only variables that are also in Swedish Data
variables = ['Country', 'League', 'Season', 'Date','Time','Home','Away','HG', 'AG', 'Res', 'PH', 'PD', 'PA', 'MaxH', 'MaxD', 'MaxA', 'AvgH', 'AvgD', 'AvgA']
merged_df = merged_df[variables]
merged_df['League'] = merged_df['League'].replace({'E1': 'Championship', 'E0': 'Premier'})

df_swe = pd.read_excel('../data/raw_data/sweden/SWE.xlsx')

merged_df_swe_eng = pd.concat([merged_df, df_swe], ignore_index=True, sort=False).drop_duplicates()
merged_df_swe_eng.dropna(subset=['Home'], inplace=True)
# To save the merged DataFrame to a CSV file, you can do the following:
merged_df_swe_eng.to_excel('../data/final_data/all_seasons.xlsx', index=False)


In [79]:
fifa_ratings = pd.read_csv("../data/raw_data/fifa/male_teams.csv")
fifa_ratings = fifa_ratings[['fifa_version', 'team_name', 'league_name', 'overall', 'attack', 'midfield', 'defence']]
fifa_ratings = fifa_ratings[(fifa_ratings['league_name']=='Premier League') | (fifa_ratings['league_name']=='Championship') | (fifa_ratings['league_name']=='Allsvenskan') | (fifa_ratings['league_name']=='Premier League') | (fifa_ratings['league_name']=='League One')]

In [80]:
fifa_ratings['Season']=fifa_ratings['fifa_version']-1+2000
fifa_ratings = fifa_ratings[['team_name','Season','overall', 'attack', 'midfield', 'defence']]

In [81]:
fifa_ratings.rename(columns={'team_name':'Home',
                            'overall':'home_overall', 
                             'attack':'home_attack', 
                             'midfield':'home_midfield', 
                             'defence':'home_defence'}, inplace=True)

In [82]:
merged_df_swe_eng_14 = merged_df_swe_eng[merged_df_swe_eng['Season']>2013]

In [83]:
home_match = merged_df_swe_eng_14.merge(fifa_ratings, on=['Home', 'Season'], how='left')

In [84]:
fifa_ratings.rename(columns={'Home':'Away',
                            'home_overall':'away_overall', 
                             'home_attack':'away_attack', 
                             'home_midfield':'away_midfield', 
                             'home_defence':'away_defence'}, inplace=True)

In [173]:
final_match = home_match.merge(fifa_ratings, on=['Away', 'Season'], how='left')

In [179]:
final_match.to_excel('../data/final_data/all_seasons_fifa.xlsx', index=False)

In [204]:
final_match = pd.read_excel('../data/final_data/all_seasons_fifa.xlsx')

In [175]:
final_match = final_match[~final_match['home_attack'].isnull()].reset_index(drop=True)

In [176]:
final_match = final_match[~final_match['away_attack'].isnull()].reset_index(drop=True)

In [177]:
final_match['home_avg_rating'] = (final_match['home_attack'] + final_match['home_midfield'] + final_match['home_defence'])/3

In [178]:
final_match['away_avg_rating'] = (final_match['away_attack'] + final_match['away_midfield'] + final_match['away_defence'])/3

In [180]:
final_match_grouped = final_match.drop_duplicates(['Season', 'League', 'Home', 'home_avg_rating']).sort_values(by=['Season', 'League', 'home_avg_rating'], ascending=[True, True, False])

In [181]:
# Luego, agrupa el DataFrame por 'Season' y 'League'
grouped = final_match_grouped.groupby(['Season', 'League'])

# Aplica la clasificación a la columna 'home_avg_rating' dentro de cada grupo
final_match_grouped['rank'] = grouped['home_avg_rating'].rank(ascending=False, method='first')

# Ahora, tu DataFrame contendrá una columna 'rank' que muestra el rango de cada equipo en cada temporada y liga según 'home_avg_rating'


In [182]:
final_match_grouped = final_match_grouped[['Home', 'Season', 'League', 'rank']].rename(columns={'rank':'home_rank'})

In [187]:
final_match_grouped.rename(columns={'Home':'Away','home_rank':'away_rank' }, inplace=True)

In [189]:
final_match = final_match.merge(final_match_grouped, on=['Away', 'Season', 'League'], how='left')

In [202]:
final_match_grouped[final_match_grouped['Away']=='Brommapojkarna']

,Away,Season,League,away_rank
8600,Brommapojkarna,2014,Allsvenskan,16.0
9572,Brommapojkarna,2018,Allsvenskan,16.0
10768,Brommapojkarna,2023,Allsvenskan,11.0


In [ ]:
final_match.to_excel('../data/final_data/all_seasons_ranking.xlsx', index=False)

In [213]:
final_match.drop_duplicates(['League']).tail(60).League.iloc[-1]

'Allsvenskan '

In [135]:

# Agrupa por 'Season', 'League', y 'Home', calcula la calificación promedio ('home_avg_rating') para cada equipo en cada temporada y liga
df_grouped = final_match_grouped.groupby(['Season', 'League', 'Home', 'home_avg_rating'])['home_avg_rating'].mean().reset_index()

# Ordena el DataFrame según 'Season', 'League', y 'home_avg_rating' de forma descendente
df_grouped.sort_values(by=['Season', 'League', 'home_avg_rating'], ascending=[True, True, False], inplace=True)

# Crea una columna 'Rank' que almacene el rango de cada equipo dentro de cada temporada y liga
df_grouped['Rank'] = df_grouped.groupby(['Season', 'League'])['home_avg_rating'].rank(ascending=False, method='min')

# Ahora df_grouped contiene la clasificación de equipos por temporada y liga, con 'Rank' indicando su posición en función de 'home_avg_rating'

# Si deseas ver el resultado:
print(df_grouped)


ValueError: cannot insert home_avg_rating, already exists

In [134]:
df_grouped

,Season,League,Home,home_avg_rating,Rank
12,2014,Allsvenskan,Malmo FF,199.0,1.0
0,2014,Allsvenskan,AIK,198.0,2.0
7,2014,Allsvenskan,Goteborg,196.0,3.0
8,2014,Allsvenskan,Hacken,193.0,4.0
4,2014,Allsvenskan,Elfsborg,191.0,5.0
...,...,...,...,...,...
602,2023,Premier,Nott'm Forest,227.0,14.0
589,2023,Premier,Bournemouth,224.0,17.0
603,2023,Premier,Sheffield United,218.0,18.0
592,2023,Premier,Burnley,216.0,19.0


In [ ]:
#### Not working

In [2]:
link = 'https://www.fifaindex.com/teams/fifa'
link_pages = []
for i in range(12,24):
    for j in range(1,24):
        link_pages.append(f'{link}{i}/?page={j}')

In [3]:
import glob
import os
import pandas as pd
import numpy as np
import re
from tqdm import tqdm
import time
from time import sleep
from datetime import datetime
tqdm.pandas()
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
import math

In [4]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
s=Service(ChromeDriverManager().install()) #MAC user might need this
driver = webdriver.Chrome(service=s)
handle = 'quinoba'
password = 'Joaquin2078'

driver.get('https://www.fifaindex.com/accounts/login/')
#driver.maximize_window()
time.sleep(3)
#login
driver.find_element(By.XPATH,'//*[@id="id_login"]').send_keys(handle)
time.sleep(3)
driver.find_element(By.XPATH, '//*[@id="gdpr-consent-tool-wrapper"]').click()
time.sleep(3)
driver.find_element(By.XPATH,'//*[@id="id_password"]').send_keys(password)
time.sleep(5)
driver.find_element(By.XPATH, '/html/body/main/div/div/div/div/div/form/div[4]/input').click()
time.sleep(3)

dfs=[]




for link in tqdm(link_pages[0:5]):
    teams = []
    leagues = []
    atts = []
    mids = []
    deffs = []
    ovrs = []
    try:
        driver.get(link)
        sleep(5)
    except:
        pass
        team = driver.find_elements(By.CLASS_NAME,"link-team")
        league = driver.find_elements(By.CLASS_NAME,"link-league")

        for team_element in team:
            team_text = team_element.text
            teams.append(team_text)
            teams = [item for item in teams if item != '']

        for league_element in league:
            league_text = league_element.text
            leagues.append(league_text)

        for t in range(1, 60):
            try:
                att =  driver.find_element(By.XPATH,f"/html/body/main/div/div/div[2]/div[2]/table/tbody/tr[{t+2}]/td[4]/span").text
                atts.append(att)
                mid = driver.find_element(By.XPATH,f"/html/body/main/div/div/div[2]/div[2]/table/tbody/tr[{t+2}]/td[5]/span").text
                mids.append(mid)
                deff = driver.find_element(By.XPATH,f"/html/body/main/div/div/div[2]/div[2]/table/tbody/tr[{t+2}]/td[6]/span").text
                deffs.append(deff)
                ovr = driver.find_element(By.XPATH,f"/html/body/main/div/div/div[2]/div[2]/table/tbody/tr[{t+2}]/td[7]/span").text
                ovrs.append(ovr)
            except:
                pass
            
    df = pd.DataFrame({'Team': teams,
                 'year': link.split('/')[4].replace('fifa', '20'),
                 'league': leagues,
                 'att_rating': atts,
                 'mid_rating': mids,
                 'def_rating': deffs,
                 'ovr_rating': ovrs})
    dfs.append(df)

ElementClickInterceptedException: Message: element click intercepted: Element <input type="submit" class="btn btn-primary w-100" value="Login"> is not clickable at point (599, 683). Other element would receive the click: <div id="gdpr-consent-tool-wrapper" tabindex="-1" aria-hidden="false" role="presentation" style="position: fixed; width: 100%; height: 100%; top: 0px; left: 0px; z-index: 2147483647; background: rgba(0, 0, 0, 0.5);">...</div>
  (Session info: chrome=118.0.5993.88)
Stacktrace:
0   chromedriver                        0x0000000104c68510 chromedriver + 4310288
1   chromedriver                        0x0000000104c604bc chromedriver + 4277436
2   chromedriver                        0x0000000104893b6c chromedriver + 293740
3   chromedriver                        0x00000001048dfcb4 chromedriver + 605364
4   chromedriver                        0x00000001048ddd7c chromedriver + 597372
5   chromedriver                        0x00000001048db9a4 chromedriver + 588196
6   chromedriv

In [6]:
a.click

<bound method WebElement.click of <selenium.webdriver.remote.webelement.WebElement (session="e9524a9cbd67b2cebf08d093d917051b", element="7D35AE4A570121FC4C32ABD803C60322_element_25")>>

In [9]:
/html/body/main/div/div/div[2]/div[2]/table/tbody/tr[3]/td[5]/span

['90',
 '86',
 '83',
 '86',
 '86',
 '84',
 '84',
 '83',
 '84',
 '82',
 '80',
 '81',
 '85',
 '82',
 '82',
 '81',
 '80',
 '82',
 '82',
 '82',
 '82',
 '81',
 '80',
 '80',
 '78',
 '73',
 '84',
 '81',
 '81',
 '79']

In [67]:
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm

def get_ratings(link_pages):
    teams = []
    years = []
    leagues = []
    atts = []
    mids = []
    deffs = []
    ovrs = []

    for link in tqdm(link_pages):
        r = requests.get(link)
        if r.status_code == 200:
            soup = BeautifulSoup(r.content, 'html.parser')
            
            # Encuentra todos los elementos con la clase 'link-team'
            team_elements = soup.find_all(class_='link-team')
            
            # Itera a través de los elementos encontrados y obtén el texto de cada uno
            for team_element in team_elements:
                team_name = team_element.get_text()
                teams.append(team_name)
                
                # Aquí puedes realizar más operaciones para obtener otros datos, si es necesario

    # Ahora, la lista 'teams' contendrá todos los textos de los elementos con la clase 'link-team'
    return teams

            

In [68]:
get_ratings(link_pages[0:2])

100%|██████████| 2/2 [00:00<00:00, 12.56it/s]


[]

In [69]:
link_pages[0]

'https://www.fifaindex.com/teams/fifa12/?page=1'

In [76]:
r = requests.get(link_pages[0])
r.status_code

403

In [77]:
r.text

'<!DOCTYPE html>\n<!--[if lt IE 7]> <html class="no-js ie6 oldie" lang="en-US"> <![endif]-->\n<!--[if IE 7]>    <html class="no-js ie7 oldie" lang="en-US"> <![endif]-->\n<!--[if IE 8]>    <html class="no-js ie8 oldie" lang="en-US"> <![endif]-->\n<!--[if gt IE 8]><!--> <html class="no-js" lang="en-US"> <!--<![endif]-->\n<head>\n<title>Attention Required! | Cloudflare</title>\n<meta charset="UTF-8" />\n<meta http-equiv="Content-Type" content="text/html; charset=UTF-8" />\n<meta http-equiv="X-UA-Compatible" content="IE=Edge" />\n<meta name="robots" content="noindex, nofollow" />\n<meta name="viewport" content="width=device-width,initial-scale=1" />\n<link rel="stylesheet" id="cf_styles-css" href="/cdn-cgi/styles/cf.errors.css" />\n<!--[if lt IE 9]><link rel="stylesheet" id=\'cf_styles-ie-css\' href="/cdn-cgi/styles/cf.errors.ie.css" /><![endif]-->\n<style>body{margin:0;padding:0}</style>\n\n\n<!--[if gte IE 10]><!-->\n<script>\n  if (!navigator.cookieEnabled) {\n    window.addEventListene

In [72]:
soup = BeautifulSoup(r.content, 'html.parser')

In [73]:
team_elements = soup.find_all(class_='link-team')

In [75]:
soup

<!DOCTYPE html>

<!--[if lt IE 7]> <html class="no-js ie6 oldie" lang="en-US"> <![endif]-->
<!--[if IE 7]>    <html class="no-js ie7 oldie" lang="en-US"> <![endif]-->
<!--[if IE 8]>    <html class="no-js ie8 oldie" lang="en-US"> <![endif]-->
<!--[if gt IE 8]><!--> <html class="no-js" lang="en-US"> <!--<![endif]-->
<head>
<title>Attention Required! | Cloudflare</title>
<meta charset="utf-8"/>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="IE=Edge" http-equiv="X-UA-Compatible"/>
<meta content="noindex, nofollow" name="robots"/>
<meta content="width=device-width,initial-scale=1" name="viewport"/>
<link href="/cdn-cgi/styles/cf.errors.css" id="cf_styles-css" rel="stylesheet"/>
<!--[if lt IE 9]><link rel="stylesheet" id='cf_styles-ie-css' href="/cdn-cgi/styles/cf.errors.ie.css" /><![endif]-->
<style>body{margin:0;padding:0}</style>
<!--[if gte IE 10]><!-->
<script>
  if (!navigator.cookieEnabled) {
    window.addEventListener('DOMContentLoaded', function